In [ ]:
drop database TourismDB;

In [ ]:

CREATE DATABASE TourismGroup8;
GO

USE TourismGroup8;
GO

-- Drop tables in reverse dependency order if they already exist
DROP TABLE IF EXISTS Booking_Item, Booking, Guide_Contract, Accommodation_Contract, Asset_Contract, Review, Booking_Schedule, Payment, Customer_Support, Itinerary, Activity, Accommodation, Schedule, Asset, Tour_Guide, Tourist_Destination, Travel_Agency, [User];

-- Create User Table with Phone_Number length constraint
CREATE TABLE [User] (
    User_ID INT IDENTITY(1,1) PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Phone_Number BIGINT CHECK (LEN(CAST(Phone_Number AS VARCHAR)) = 10),
    Email_ID VARCHAR(100) NOT NULL UNIQUE,
    Password VARCHAR(100) NOT NULL,
    Street VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    Pincode INT CHECK (Pincode BETWEEN 100000 AND 999999)
);

-- Create Travel Agency Table
CREATE TABLE Travel_Agency (
    Agency_ID INT IDENTITY(1,1) PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Location VARCHAR(100),
    Contact_Info BIGINT CHECK (Contact_Info > 999999999 AND Contact_Info <= 9999999999),
    Rating INT CHECK (Rating BETWEEN 1 AND 5)
);

-- Create Tourist Destination Table
CREATE TABLE Tourist_Destination (
    Destination_ID INT IDENTITY(1,1) PRIMARY KEY,
    Agency_ID INT NOT NULL,
    Name VARCHAR(100) NOT NULL,
    Location VARCHAR(100),
    Popular_Attractions VARCHAR(500),
    Rating INT CHECK (Rating BETWEEN 1 AND 5),
    Description VARCHAR(500),
    FOREIGN KEY (Agency_ID) REFERENCES Travel_Agency(Agency_ID)
);

-- Create Itinerary Table
CREATE TABLE Itinerary (
    Itinerary_ID INT IDENTITY(1,1) PRIMARY KEY,
    User_ID INT NOT NULL,
    Start_Date DATETIME NOT NULL,
    End_Date DATETIME NOT NULL,
    Destination VARCHAR(100),
    Total_Cost FLOAT CHECK (Total_Cost >= 0),
    FOREIGN KEY (User_ID) REFERENCES [User](User_ID),
    CHECK (End_Date >= Start_Date)
);

-- Create Customer Support Table
CREATE TABLE Customer_Support (
    Support_ID INT IDENTITY(1,1) PRIMARY KEY,
    User_ID INT NOT NULL,
    Issue_Description VARCHAR(500) NOT NULL,
    Resolution_Status VARCHAR(50) CHECK (Resolution_Status IN ('Resolved', 'Pending', 'Closed')),
    Resolution_Date DATETIME,
    FOREIGN KEY (User_ID) REFERENCES [User](User_ID)
);

-- Create Activity Table
CREATE TABLE Activity (
    Activity_ID INT IDENTITY(1,1) PRIMARY KEY,
    Destination_ID INT NOT NULL,
    Name VARCHAR(100) NOT NULL,
    Description VARCHAR(500),
    Location VARCHAR(100),
    Duration INT CHECK (Duration > 0),
    Price INT CHECK (Price >= 0),
    FOREIGN KEY (Destination_ID) REFERENCES Tourist_Destination(Destination_ID)
);

-- Create Accommodation Table
CREATE TABLE Accommodation (
    Accommodation_ID INT IDENTITY(1,1) PRIMARY KEY,
    Destination_ID INT NOT NULL,
    Name VARCHAR(100) NOT NULL,
    Street VARCHAR(100),
    City VARCHAR(50),
    State VARCHAR(50),
    Zip_Code INT,
    Type VARCHAR(50),
    Price_Per_Night FLOAT CHECK (Price_Per_Night >= 0),
    Availability_Status VARCHAR(50) CHECK (Availability_Status IN ('Available', 'Booked')),
    Rating INT CHECK (Rating BETWEEN 1 AND 5),
    FOREIGN KEY (Destination_ID) REFERENCES Tourist_Destination(Destination_ID)
);

-- Create Assets Table to represent transport and other resources
CREATE TABLE Asset (
    Asset_ID INT IDENTITY(1,1) PRIMARY KEY,
    Type VARCHAR(50) NOT NULL,
    Company_Name VARCHAR(100),
    Description VARCHAR(500),
    Value FLOAT CHECK (Value >= 0)
);

-- Create Schedule Table for transport and other scheduled resources
CREATE TABLE Schedule (
    Schedule_ID INT IDENTITY(1,1) PRIMARY KEY,
    Asset_ID INT NOT NULL,
    Departure_Time DATETIME NOT NULL,
    Arrival_Time DATETIME NOT NULL,
    Price FLOAT CHECK (Price >= 0),
    FOREIGN KEY (Asset_ID) REFERENCES Asset(Asset_ID),
    CHECK (Arrival_Time >= Departure_Time)
);

-- Create Tour Guide Table with Phone Number constraint
CREATE TABLE Tour_Guide (
    Guide_ID INT IDENTITY(1,1) PRIMARY KEY,
    Name VARCHAR(100) NOT NULL,
    Language VARCHAR(50),
    Experience_Years INT CHECK (Experience_Years >= 0),
    Contact_Info BIGINT CHECK (LEN(CAST(Contact_Info AS VARCHAR)) = 10),
    Rating INT CHECK (Rating BETWEEN 1 AND 5)
);

-- Create Review Table
CREATE TABLE Review (
    Review_ID INT IDENTITY(1,1) PRIMARY KEY,
    User_ID INT NOT NULL,
    Destination_ID INT NOT NULL,
    Accommodation_ID INT,
    Activity_ID INT,
    Rating INT CHECK (Rating BETWEEN 1 AND 5),
    Review_Text VARCHAR(500),
    Review_Date DATETIME DEFAULT GETDATE(),
    FOREIGN KEY (User_ID) REFERENCES [User](User_ID),
    FOREIGN KEY (Destination_ID) REFERENCES Tourist_Destination(Destination_ID),
    FOREIGN KEY (Accommodation_ID) REFERENCES Accommodation(Accommodation_ID),
    FOREIGN KEY (Activity_ID) REFERENCES Activity(Activity_ID)
);

-- Create Booking Table with UNIQUE constraint on Itinerary_ID
CREATE TABLE Booking (
    Booking_ID INT IDENTITY(1,1) PRIMARY KEY,
    Itinerary_ID INT NOT NULL UNIQUE,
    User_ID INT NOT NULL,
    Booking_Date DATETIME DEFAULT GETDATE(),
    Total_Cost FLOAT CHECK (Total_Cost >= 0),
    Payment_Status VARCHAR(50) CHECK (Payment_Status IN ('Paid', 'Pending', 'Canceled')),
    Accommodation_ID INT NULL,   -- New field added
    Activity_ID INT NULL,        -- New field added
    Schedule_ID INT NULL,        -- New field added
    FOREIGN KEY (User_ID) REFERENCES [User](User_ID),
    FOREIGN KEY (Itinerary_ID) REFERENCES Itinerary(Itinerary_ID),
    FOREIGN KEY (Accommodation_ID) REFERENCES Accommodation(Accommodation_ID),
    FOREIGN KEY (Activity_ID) REFERENCES Activity(Activity_ID),
    FOREIGN KEY (Schedule_ID) REFERENCES Schedule(Schedule_ID)
);

-- Create Payment Table
CREATE TABLE Payment (
    Payment_ID INT IDENTITY(1,1) PRIMARY KEY,
    Booking_ID INT NOT NULL,
    Payment_Method VARCHAR(50) CHECK (Payment_Method IN ('Credit Card', 'Debit Card', 'UPI', 'Cash')),
    Payment_Amount FLOAT CHECK (Payment_Amount > 0),
    Payment_Status VARCHAR(50) CHECK (Payment_Status IN ('Paid', 'Pending', 'Failed')),
    Transaction_Date DATETIME DEFAULT GETDATE(),
    FOREIGN KEY (Booking_ID) REFERENCES Booking(Booking_ID)
);

-- Create Booking_Schedule Table to track asset schedules within a booking
CREATE TABLE Booking_Schedule (
    Schedule_ID INT NOT NULL,
    Booking_ID INT NOT NULL,
    StartDate DATETIME NOT NULL,
    EndDate DATETIME NOT NULL,
    PRIMARY KEY (Schedule_ID, Booking_ID),
    FOREIGN KEY (Schedule_ID) REFERENCES Schedule(Schedule_ID),
    FOREIGN KEY (Booking_ID) REFERENCES Booking(Booking_ID),
    CHECK (EndDate >= StartDate)
);

-- Create Guide Contract Table
CREATE TABLE Guide_Contract (
    Activity_ID INT NOT NULL,
    Guide_ID INT NOT NULL,
    ContractStartDate DATETIME NOT NULL,
    ContractEndDate DATETIME NOT NULL,
    PRIMARY KEY (Activity_ID, Guide_ID),
    FOREIGN KEY (Activity_ID) REFERENCES Activity(Activity_ID),
    FOREIGN KEY (Guide_ID) REFERENCES Tour_Guide(Guide_ID),
    CHECK (ContractEndDate >= ContractStartDate)
);

-- Create Accommodation Contract Table
CREATE TABLE Accommodation_Contract (
    Agency_ID INT NOT NULL,
    Accommodation_ID INT NOT NULL,
    ContractStartDate DATETIME NOT NULL,
    ContractEndDate DATETIME NOT NULL,
    PRIMARY KEY (Agency_ID, Accommodation_ID),
    FOREIGN KEY (Agency_ID) REFERENCES Travel_Agency(Agency_ID),
    FOREIGN KEY (Accommodation_ID) REFERENCES Accommodation(Accommodation_ID),
    CHECK (ContractEndDate >= ContractStartDate)
);

-- Create Asset Contract Table (replaces Transport_Contract)
CREATE TABLE Asset_Contract (
    Agency_ID INT NOT NULL,
    Asset_ID INT NOT NULL,
    ContractStartDate DATETIME NOT NULL,
    ContractEndDate DATETIME NOT NULL,
    PRIMARY KEY (Agency_ID, Asset_ID),
    FOREIGN KEY (Agency_ID) REFERENCES Travel_Agency(Agency_ID),
    FOREIGN KEY (Asset_ID) REFERENCES Asset(Asset_ID),
    CHECK (ContractEndDate >= ContractStartDate)
);

In [ ]:
-- User Table (10 Records)
INSERT INTO [User] (Name, Phone_Number, Email_ID, Password, Street, City, State, Pincode) VALUES
('Alice Smith', 9876543210, 'alice.smith@example.com', 'password123', '123 Main St', 'New York', 'NY', 100001),
('Bob Johnson', 8765432109, 'bob.johnson@example.com', 'securePass', '456 Oak St', 'Los Angeles', 'CA', 900001),
('Carol White', 7654321098, 'carol.white@example.com', 'myPassword', '789 Pine St', 'Chicago', 'IL', 600001),
('David Brown', 6543210987, 'david.brown@example.com', 'pass1234', '321 Elm St', 'Houston', 'TX', 700001),
('Emma Black', 5432109876, 'emma.black@example.com', 'black123', '123 Maple St', 'Miami', 'FL', 800001),
('Frank Green', 4321098765, 'frank.green@example.com', 'green123', '456 Cedar St', 'Boston', 'MA', 200001),
('Grace Lee', 3210987654, 'grace.lee@example.com', 'lee12345', '789 Spruce St', 'Seattle', 'WA', 980001),
('Hank King', 2109876543, 'hank.king@example.com', 'king123', '111 Ash St', 'Austin', 'TX', 733010),
('Ivy Knight', 1098765432, 'ivy.knight@example.com', 'knight2021', '222 Birch St', 'Denver', 'CO', 802001),
('Jack Frost', 9876543211, 'jack.frost@example.com', 'winterpass', '333 Redwood St', 'San Francisco', 'CA', 941001);

-- Travel Agency Table (10 Records)
INSERT INTO Travel_Agency (Name, Location, Contact_Info, Rating) VALUES
('Wanderlust Travel', 'New York', 9988776655, 5),
('Explore More', 'Los Angeles', 8877665544, 4),
('Adventure Time', 'Chicago', 7766554433, 3),
('Global Excursions', 'Miami', 6655443322, 5),
('Getaway Experts', 'Houston', 5544332211, 4),
('Dream Destinations', 'Boston', 4433221100, 3),
('Travel Masters', 'Seattle', 3322110099, 4),
('Exciting Escapes', 'Austin', 2211009988, 5),
('Holiday Planners', 'Denver', 1100998877, 4),
('World Wanderers', 'San Francisco', 9998887766, 5);

-- Tourist Destination Table (10 Records)
INSERT INTO Tourist_Destination (Agency_ID, Name, Location, Popular_Attractions, Rating, Description) VALUES
(1, 'Grand Canyon', 'Arizona', 'Hiking, Scenic Views', 5, 'A breathtaking natural wonder.'),
(2, 'Statue of Liberty', 'New York', 'Historic Sites, Museum', 4, 'Iconic symbol of freedom.'),
(3, 'Disneyland', 'California', 'Theme Park, Rides', 5, 'The happiest place on earth.'),
(4, 'Niagara Falls', 'New York', 'Waterfall, Tours', 5, 'Stunning waterfalls.'),
(5, 'Yellowstone Park', 'Wyoming', 'Wildlife, Geysers', 5, 'Beautiful park with geothermal features.'),
(6, 'Hollywood', 'Los Angeles', 'Walk of Fame, Studios', 4, 'Entertainment capital.'),
(7, 'Yosemite National Park', 'California', 'Hiking, Climbing', 5, 'Majestic natural landscapes.'),
(8, 'Mount Rushmore', 'South Dakota', 'Historic Landmark', 4, 'Iconic presidential carvings.'),
(9, 'Times Square', 'New York', 'Shopping, Theaters', 4, 'Popular commercial hub.'),
(10, 'Golden Gate Bridge', 'San Francisco', 'Scenic Views', 5, 'Famous suspension bridge.');


-- Itinerary Table (10 Records)
INSERT INTO Itinerary (User_ID, Start_Date, End_Date, Destination, Total_Cost) VALUES
(1, '2023-12-01', '2023-12-10', 'Grand Canyon', 1200.50),
(2, '2023-11-15', '2023-11-20', 'Statue of Liberty', 800.75),
(3, '2023-12-25', '2024-01-05', 'Disneyland', 1500.00),
(4, '2024-01-10', '2024-01-15', 'Niagara Falls', 950.00),
(5, '2024-02-01', '2024-02-05', 'Yellowstone Park', 1300.00),
(6, '2024-03-15', '2024-03-20', 'Hollywood', 850.00),
(7, '2024-04-10', '2024-04-15', 'Yosemite National Park', 1250.00),
(8, '2024-05-01', '2024-05-10', 'Mount Rushmore', 700.00),
(9, '2024-06-01', '2024-06-05', 'Times Square', 600.00),
(10, '2024-07-01', '2024-07-10', 'Golden Gate Bridge', 1100.00);

-- Customer Support Table (10 Records)
INSERT INTO Customer_Support (User_ID, Issue_Description, Resolution_Status, Resolution_Date) VALUES
(1, 'Booking issue', 'Resolved', '2023-11-15'),
(2, 'Payment failure', 'Pending', NULL),
(3, 'Account login issue', 'Closed', '2023-10-10'),
(4, 'Itinerary update', 'Resolved', '2023-09-05'),
(5, 'Accommodation complaint', 'Pending', NULL),
(6, 'Guide language issue', 'Resolved', '2023-08-20'),
(7, 'Schedule delay', 'Closed', '2023-07-15'),
(8, 'Transportation issue', 'Resolved', '2023-06-05'),
(9, 'Refund request', 'Pending', NULL);


INSERT INTO Activity (Destination_ID, Name, Description, Location, Duration, Price) VALUES
(1, 'Canyon Hike', 'Guided hiking tour of the canyon.', 'South Rim', 5, 100),
(2, 'Liberty Island Tour', 'Ferry and guided tour of Liberty Island.', 'Liberty Island', 2, 50),
(3, 'Disneyland Day Pass', 'Full day pass for Disneyland.', 'Anaheim', 8, 150),
(4, 'Falls Boat Ride', 'Boat ride at Niagara Falls.', 'Niagara Falls', 1, 40),
(5, 'Wildlife Safari', 'Guided safari in Yellowstone.', 'Yellowstone Park', 3, 120),
(6, 'Studio Tour', 'Hollywood studios tour.', 'Hollywood', 2, 80),
(7, 'Rock Climbing', 'Climbing activity in Yosemite.', 'Yosemite National Park', 4, 60),
(8, 'Rushmore Tour', 'Guided tour of Mount Rushmore.', 'Mount Rushmore', 2, 50),
(9, 'Broadway Show', 'Show tickets in Times Square.', 'Times Square', 3, 100),
(10, 'Bridge Walk', 'Golden Gate Bridge walking tour.', 'Golden Gate Bridge', 2, 30);

-- Accommodation Table (10 Records)
INSERT INTO Accommodation (Destination_ID, Name, Street, City, State, Zip_Code, Type, Price_Per_Night, Availability_Status, Rating) VALUES
(1, 'Canyon Lodge', '123 Canyon Rd', 'Flagstaff', 'AZ', 86001, 'Hotel', 120.00, 'Available', 4),
(2, 'Liberty Inn', '45 Liberty St', 'New York', 'NY', 10001, 'Hotel', 200.00, 'Booked', 5),
(3, 'Disney Resort', '789 Disney Ln', 'Anaheim', 'CA', 92802, 'Resort', 300.00, 'Available', 5),
(4, 'Falls Hotel', '111 Falls Rd', 'Buffalo', 'NY', 14201, 'Hotel', 150.00, 'Booked', 4),
(5, 'Yellowstone Cabins', '234 Park Ave', 'Cody', 'WY', 82414, 'Cabin', 100.00, 'Available', 5),
(6, 'Hollywood Suites', '678 Fame Rd', 'Los Angeles', 'CA', 90001, 'Hotel', 180.00, 'Booked', 4),
(7, 'Yosemite Lodge', '345 Mountain Rd', 'El Portal', 'CA', 95318, 'Lodge', 140.00, 'Available', 5),
(8, 'Rushmore Inn', '567 Monument Rd', 'Keystone', 'SD', 57751, 'Inn', 130.00, 'Available', 4),
(9, 'Broadway Hotel', '890 Theater St', 'New York', 'NY', 10036, 'Hotel', 250.00, 'Booked', 5),
(10, 'Golden Gate Inn', '456 Golden Rd', 'San Francisco', 'CA', 94101, 'Inn', 160.00, 'Available', 4);

-- Asset Table (10 Records)
INSERT INTO Asset (Type, Company_Name, Description, Value) VALUES
('Bus', 'TravelX Transport', 'Tourist bus for group travel.', 50000),
('Van', 'City Van Rentals', 'Van for city tours and short trips.', 30000),
('Car', 'Elite Car Rentals', 'Luxury car for VIPs.', 25000),
('Boat', 'Marine Adventures', 'Boat for river tours.', 45000),
('Helicopter', 'Sky High Tours', 'Helicopter for aerial tours.', 150000),
('Bike', 'Bike Rental Co.', 'Bikes for mountain trail.', 5000),
('Train', 'Regional Rail', 'Train seats for cross-country travel.', 80000),
('SUV', 'All-Terrain Rentals', 'SUV for off-road adventures.', 40000),
('Plane', 'Air Wings', 'Small charter plane for tours.', 200000),
('Kayak', 'Water Sports Co.', 'Kayak for water adventure.', 1000);

-- Schedule Table (10 Records)
INSERT INTO Schedule (Asset_ID, Departure_Time, Arrival_Time, Price) VALUES
(1, '2024-01-10 08:00:00', '2024-01-10 12:00:00', 100.00),
(2, '2024-01-15 09:00:00', '2024-01-15 12:00:00', 50.00),
(3, '2024-01-20 14:00:00', '2024-01-20 17:00:00', 200.00),
(4, '2024-01-25 08:00:00', '2024-01-25 10:00:00', 75.00),
(5, '2024-02-01 13:00:00', '2024-02-01 15:00:00', 300.00),
(6, '2024-02-05 07:00:00', '2024-02-05 08:00:00', 20.00),
(7, '2024-02-10 06:00:00', '2024-02-10 11:00:00', 150.00),
(8, '2024-02-15 10:00:00', '2024-02-15 14:00:00', 180.00),
(9, '2024-02-20 12:00:00', '2024-02-20 13:30:00', 100.00),
(10, '2024-02-25 08:00:00', '2024-02-25 09:00:00', 30.00);





INSERT INTO Tour_Guide (Name, Language, Experience_Years, Contact_Info, Rating) VALUES
('John Doe', 'English', 5, 9876543210, 4),
('Sara Lee', 'Spanish', 8, 8765432109, 5),
('Raj Patel', 'Hindi', 3, 7654321098, 4),
('Anna Lopez', 'English', 6, 6543210987, 5),
('Chris Brown', 'French', 2, 5432109876, 4),
('Emily Wang', 'Mandarin', 7, 4321098765, 5),
('Luis Martinez', 'Spanish', 4, 3210987654, 4),
('Sophia Zhang', 'Mandarin', 3, 2109876543, 4),
('Omar Hassan', 'Arabic', 6, 1098765432, 5),
('Tommy Ng', 'English', 9, 9876543211, 5);


-- Review Table (10 Records)
INSERT INTO Review (User_ID, Destination_ID, Accommodation_ID, Activity_ID, Rating, Review_Text, Review_Date) VALUES
(1, 1, 1, 1, 5, 'Amazing experience!', '2023-12-10'),
(2, 2, 2, 2, 4, 'Good tour but could be better.', '2023-11-20'),
(3, 3, 3, 3, 5, 'Best holiday experience!', '2024-01-05'),
(4, 4, 4, 4, 5, 'Absolutely fantastic!', '2024-01-15'),
(5, 5, 5, 5, 4, 'Great but expensive.', '2024-02-05'),
(6, 6, 6, 6, 4, 'Enjoyed the tour a lot.', '2024-03-20'),
(7, 7, 7, 7, 5, 'Incredible views!', '2024-04-15'),
(8, 8, 8, 8, 4, 'Loved it, highly recommend.', '2024-05-10'),
(9, 9, 9, 9, 5, 'Unforgettable experience.', '2024-06-05'),
(10, 10, 10, 10, 5, 'Perfect holiday!', '2024-07-10');


-- Booking Table (10 Records)
INSERT INTO Booking (Itinerary_ID, User_ID, Booking_Date, Total_Cost, Payment_Status, Accommodation_ID, Activity_ID, Schedule_ID) VALUES
(1, 1, '2023-12-01', 1200.50, 'Paid', 1, 1, 1),
(2, 2, '2023-11-15', 800.75, 'Pending', 2, 2, 2),
(3, 3, '2023-12-25', 1500.00, 'Paid', 3, 3, 3),
(4, 4, '2024-01-10', 950.00, 'Paid', 4, 4, 4),
(5, 5, '2024-02-01', 1300.00, 'Pending', 5, 5, 5),
(6, 6, '2024-03-15', 850.00, 'Paid', 6, 6, 6),
(7, 7, '2024-04-10', 1250.00, 'Paid', 7, 7, 7),
(8, 8, '2024-05-01', 700.00, 'Pending', 8, 8, 8),
(9, 9, '2024-06-01', 600.00, 'Paid', 9, 9, 9),
(10, 10, '2024-07-01', 1100.00, 'Pending', 10, 10, 10);








-- Payment Table (10 Records)
INSERT INTO Payment (Booking_ID, Payment_Method, Payment_Amount, Payment_Status, Transaction_Date) VALUES
(1, 'Credit Card', 1200.50, 'Paid', '2023-12-01'),
(2, 'UPI', 800.75, 'Pending', '2023-11-15'),
(3, 'Debit Card', 1500.00, 'Paid', '2023-12-25'),
(4, 'Cash', 950.00, 'Paid', '2024-01-10'),
(5, 'Credit Card', 1300.00, 'Pending', '2024-02-01'),
(6, 'UPI', 850.00, 'Paid', '2024-03-15'),
(7, 'Debit Card', 1250.00, 'Paid', '2024-04-10'),
(8, 'Cash', 700.00, 'Pending', '2024-05-01'),
(9, 'Credit Card', 600.00, 'Paid', '2024-06-01'),
(10, 'UPI', 1100.00, 'Pending', '2024-07-01');









-- Booking Schedule Table (10 Records)
INSERT INTO Booking_Schedule (Schedule_ID, Booking_ID, StartDate, EndDate) VALUES
(1, 1, '2024-01-10', '2024-01-11'),
(2, 2, '2024-01-15', '2024-01-16'),
(3, 3, '2024-01-20', '2024-01-21'),
(4, 4, '2024-01-25', '2024-01-26'),
(5, 5, '2024-02-01', '2024-02-02'),
(6, 6, '2024-02-05', '2024-02-06'),
(7, 7, '2024-02-10', '2024-02-11'),
(8, 8, '2024-02-15', '2024-02-16'),
(9, 9, '2024-02-20', '2024-02-21'),
(10, 10, '2024-02-25', '2024-02-26');

-- Guide Contract Table (10 Records)
INSERT INTO Guide_Contract (Activity_ID, Guide_ID, ContractStartDate, ContractEndDate) VALUES
(1, 1, '2024-01-10', '2024-01-12'),
(2, 2, '2024-01-15', '2024-01-17'),
(3, 3, '2024-01-20', '2024-01-22'),
(4, 4, '2024-01-25', '2024-01-27'),
(5, 5, '2024-02-01', '2024-02-03'),
(6, 6, '2024-02-05', '2024-02-07'),
(7, 7, '2024-02-10', '2024-02-12'),
(8, 8, '2024-02-15', '2024-02-17'),
(9, 9, '2024-02-20', '2024-02-22'),
(10, 10, '2024-02-25', '2024-02-27');

-- Accommodation Contract Table (10 Records)
INSERT INTO Accommodation_Contract (Agency_ID, Accommodation_ID, ContractStartDate, ContractEndDate) VALUES
(1, 1, '2024-01-01', '2024-12-31'),
(2, 2, '2024-01-01', '2024-12-31'),
(3, 3, '2024-01-01', '2024-12-31'),
(4, 4, '2024-01-01', '2024-12-31'),
(5, 5, '2024-01-01', '2024-12-31'),
(6, 6, '2024-01-01', '2024-12-31'),
(7, 7, '2024-01-01', '2024-12-31'),
(8, 8, '2024-01-01', '2024-12-31'),
(9, 9, '2024-01-01', '2024-12-31'),
(10, 10, '2024-01-01', '2024-12-31');

-- Asset Contract Table (10 Records)
INSERT INTO Asset_Contract (Agency_ID, Asset_ID, ContractStartDate, ContractEndDate) VALUES
(1, 1, '2024-01-01', '2024-12-31'),
(2, 2, '2024-01-01', '2024-12-31'),
(3, 3, '2024-01-01', '2024-12-31'),
(4, 4, '2024-01-01', '2024-12-31'),
(5, 5, '2024-01-01', '2024-12-31'),
(6, 6, '2024-01-01', '2024-12-31'),
(7, 7, '2024-01-01', '2024-12-31'),
(8, 8, '2024-01-01', '2024-12-31'),
(9, 9, '2024-01-01', '2024-12-31'),
(10, 10, '2024-01-01', '2024-12-31');

